In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = False

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSR%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFDSR%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempF2S = InterfaceData(size=inodes, value=300.0)
TempS = InterfaceData(size=inodes, value=T_bot)
tempF = InterfaceData(size=inodes, value=300.0)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
Nmax = 8000
t = 0.0
avg_pc_iterations = 0.0
flog = open('FDSR%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FDSR%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    # mapping
    tempF.curr[:] = TempS.curr[s2f]
    
    solverF.put_temperature_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=tempF.curr
    )
    
    while True:
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        abTempF2S.curr[:] = abTempF.curr[f2s]
        # update solid flux interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        TempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
        # mapping
        tempF.curr[:] = TempS.curr[s2f]
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(tempF)
                under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-300)/10
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=14.
step=1, pc_iterations=13.
step=2, pc_iterations=13.
step=3, pc_iterations=13.
step=4, pc_iterations=12.
step=5, pc_iterations=13.
step=6, pc_iterations=12.
step=7, pc_iterations=12.
step=8, pc_iterations=12.
step=9, pc_iterations=12.
step=10, pc_iterations=12.
step=11, pc_iterations=11.
step=12, pc_iterations=12.
step=13, pc_iterations=11.
step=14, pc_iterations=11.
step=15, pc_iterations=10.
step=16, pc_iterations=10.
step=17, pc_iterations=10.
step=18, pc_iterations=10.
step=19, pc_iterations=10.
step=20, pc_iterations=10.
step=21, pc_iterations=10.
step=22, pc_iterations=10.
step=23, pc_iterations=10.
step=24, pc_iterations=10.
step=25, pc_iterations=10.
step=26, pc_iterations=10.
step=27, pc_iterations=7.
step=28, pc_iterations=7.
step=29, pc_iterations=7.
step=30, pc_iterations=7.
step=31, pc_iterations=7.
step=32, pc_iterations=7.
step=33, pc_iterations=7.
step=34, pc_iterations=7.
step=35, pc_iterations=6.
step=36, pc_iterations=6.
step=37, pc_iteration

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFDSR%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.93475252 0.93609379 0.93829424 0.9404099  0.94236744 0.94413458
 0.94578405 0.94729086 0.94870646 0.95007918 0.9513424  0.95256842
 0.95371008 0.95481213 0.95585378 0.95685397 0.95779945 0.95871759
 0.95958716 0.96042147 0.96122085 0.96198044 0.96272428 0.96340916
 0.9641045  0.96472679 0.96536351 0.96594934 0.96652808 0.96707751
 0.96759568 0.96810928 0.96858996 0.9690502  0.96950854 0.96991761
 0.97035187 0.9707229  0.9711147  0.97147323 0.97182003 0.97215783
 0.97247233 0.97277943 0.97307148 0.97334908 0.97361695 0.97387408
 0.97410686 0.97435861 0.97455437 0.97478216 0.97497265 0.97515995
 0.97534322 0.9754979  0.97566424 0.97580124 0.97593976 0.97606642
 0.97617099 0.97629568 0.97636582 0.97646981 0.97653305 0.97659932
 0.9766566  0.97668906 0.9767272  0.97674574 0.97674689 0.97675235
 0.97671985 0.97671391 0.97664836 0.9766133  0.9765339  0.97645456
 0.97636134 0.97623758 0.97611662 0.97596361 0.97579549 0.97561132
 0.97540044 0.97517581 0.97491494 0.97464222 0.97433225 0.9739